In [ ]:
import bitalino
import numpy
import time
import subprocess
import pygame
import math

# Adresse MAC du capteur
macAddress = "98:D3:11:FE:03:67"

# Tentative de connexion au capteur BITalino
try:
    device = bitalino.BITalino(macAddress)
    time.sleep(1)
    print("Capteur BITalino connecté")
except Exception as e:
    print(f"Erreur de connexion au capteur: {e}")
    exit()

# Paramètres de lecture
srate = 1000
nframes = 100
threshold = 5

# Démarrage de l'acquisition des données
try:
    device.start(srate, [0])
    print("START")
except Exception as e:
    print(f"Erreur lors du démarrage de l'acquisition: {e}")
    exit()

# Initialisation de Pygame
pygame.init()

# Définir la taille de la fenêtre
window_size = (800, 600)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Données EDA - Interface Futuriste")

# Police pour afficher les données
font = pygame.font.SysFont("Arial", 30)
hud_font = pygame.font.SysFont("Press Start 2P", 25)

# Couleurs
BACKGROUND_COLOR = (20, 20, 20)  # Fond sombre
TEXT_COLOR = (0, 255, 0)  # Texte vert fluo
HIGHLIGHT_COLOR = (255, 0, 0)  # Alerte rouge
NEON_BLUE = (0, 255, 255)  # Bleu néon
NEON_PURPLE = (255, 0, 255)  # Violet néon

# Variables pour stocker la dernière valeur EDA
last_EDA_value = 0.0

# Fonction pour dessiner le fond dynamique (avec effet de vague)
def draw_background():
    screen.fill(BACKGROUND_COLOR)
    
    # Effet de vague
    for i in range(0, window_size[0], 10):
        wave_height = math.sin(i * 0.05 + time.time()) * 20
        pygame.draw.line(screen, NEON_BLUE, (i, window_size[1] - 100), (i, window_size[1] - 100 - wave_height), 2)

# Fonction d'animation d'un cercle dynamique
def draw_dynamic_circle(EDA_value):
    radius = int(50 + (EDA_value / 100) * 100)  # Le rayon change en fonction de la valeur EDA
    pygame.draw.circle(screen, NEON_PURPLE, (window_size[0] // 2, window_size[1] // 2), radius)
    pygame.draw.circle(screen, (0, 0, 0), (window_size[0] // 2, window_size[1] // 2), radius - 10)

# Afficher l'interface HUD de manière élégante
def draw_hud(EDA_value):
    # Titre
    title_text = hud_font.render("CAPTEUR EDA", True, TEXT_COLOR)
    screen.blit(title_text, (30, 30))

    # Valeur EDA actuelle
    EDA_text = font.render(f"EDA: {EDA_value:.2f}", True, TEXT_COLOR)
    screen.blit(EDA_text, (30, 100))

    # Effet de barre de progression
    bar_width = int((EDA_value / 100) * (window_size[0] - 60))
    pygame.draw.rect(screen, NEON_BLUE, pygame.Rect(30, 200, bar_width, 30))

    # Afficher un indicateur de seuil
    if EDA_value > threshold:
        threshold_text = hud_font.render("ALERTE: Seuil franchi!", True, HIGHLIGHT_COLOR)
        screen.blit(threshold_text, (30, 250))

# Boucle principale pour afficher les données en temps réel
running = True
while running:
    # Gérer les événements Pygame (fermeture de la fenêtre, etc.)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    try:
        # Lire les données du capteur
        data = device.read(nframes)

        # Si la moyenne est inférieure à 1, on arrête la lecture
        if numpy.mean(data[:, 1]) < 1:
            break

        EDA = data[:, -1]
        last_EDA_value = EDA[-1]  # Dernière valeur EDA

        # Calcul de l'enveloppe
        envelope = numpy.mean(abs(numpy.diff(EDA)))

        # Si l'enveloppe dépasse le seuil, on envoie un trigger
        if envelope > threshold:
            device.trigger([0, 1])
        else:
            device.trigger([0, 0])

    except Exception as e:
        print(f"Erreur de lecture des données : {e}")
        break

    # Dessiner le fond, les cercles dynamiques et l'interface
    draw_background()
    draw_dynamic_circle(last_EDA_value)
    draw_hud(last_EDA_value)

    # Mettre à jour l'affichage
    pygame.display.flip()

    # Attendre un peu avant de relire les données
    pygame.time.delay(100)

# Arrêter le capteur et fermer Pygame proprement
print("STOP")
device.trigger([0, 0])
device.stop()
device.close()

pygame.quit()


Capteur BITalino connecté
START
STOP
